# PROGRAMA PARA CALCULAR LAS MÉTRICAS A PARTIR DE UN JSON

In [82]:
import json
import os
import datetime 

In [35]:
dirNames = ['Login', 'Logout', 'LoginZone', 'LogoutZone', 'OpenChest', 'OpenInv', 'OpenShop', 'OpenSkills', 'TutoTask', 'UsePot', 'UseSkill']

### Comprobación de los eventos

In [64]:
def getEventsFromPlayer(player):
    initial_count = 0
    
    dict_JSONS = {
        "Login": [],
        "Logout": [],
        "LoginZone": [],
        "LogoutZone": [],
        "OpenChest": [],
        "OpenInv": [],
        "OpenShop": [],
        "OpenSkills": [],
        "TutoTask": [],
        "UsePot": [],
        "UseSkill": [],
    }
    
    for directory in dirNames:
        if os.path.exists(directory + "/" + player) and os.path.isdir(directory + "/" + player):
            if not os.listdir(directory + "/" + player):
                print(f"No hay eventos de {directory}.")
            else:    
                print(f"Hay eventos de {directory}.")
                for path in os.listdir(directory + "/" + player):
                    if os.path.isfile(os.path.join(directory + "/" + player, path)):
                        f = open(directory + "/" + player + "/" + directory + str(initial_count) + ".json", "r")
                        content = f.read()
                        jsondecoded = json.loads(content)
                        dict_JSONS[directory].insert(initial_count, jsondecoded)
                        initial_count += 1
            print(initial_count)
            initial_count = 0
        else:
            print(f"El directorio {directory} no existe.")
    return dict_JSONS

In [81]:
dict_JSONS0 = getEventsFromPlayer("Player0")

Hay eventos de Login.
3
Hay eventos de Logout.
3
Hay eventos de LoginZone.
13
Hay eventos de LogoutZone.
12
Hay eventos de OpenChest.
29
Hay eventos de OpenInv.
34
Hay eventos de OpenShop.
6
Hay eventos de OpenSkills.
6
Hay eventos de TutoTask.
5
El directorio UsePot no existe.
Hay eventos de UseSkill.
56


## Método para calcular el número de muertes del jugador pasado por parámetro.

In [76]:
def getNumDeaths(dict_JSON):
    cont_muertes = 0
    for zone in dict_JSON["LogoutZone"]:
        if zone["zone"] == -1:
            cont_muertes += 1
    return cont_muertes

## Método para calcular tiempo que tarda un jugador en pasar la primera zona

In [99]:
def getTimeZone(dict_JSON):
    timeIni = 0
    timeOut = 0
    result = 0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            date_timeIni = datetime.datetime.fromtimestamp(timeIni)
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    result = date_timeOut - date_timeIni
                    return result

In [98]:
getTimeZone(dict_JSONS0)

datetime.timedelta(seconds=105)

## Método para calcular el número de veces que se abre el inventario durante la primera zona por cada jugador

In [107]:
def getNumOpenInv(dict_JSON):
    cont_open = 0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"]== 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    for openInv in dict_JSON["OpenInv"]:
                        timeOpenInv=openInv["timestamp"]
                        if timeOpenInv > timeIni and timeOpenInv < timeOut :
                            cont_open+=1
    return cont_open

## Método para calcular si el jugador ha abierto el inventario poco después de abrir un cofre

In [103]:
def getNumOpenInvAfterChest(dict_JSON,timeToOpenChest):
    for openChest in dict_JSON["OpenChest"]:
            timeIni = openChest["timestamp"]
            for openInv in dict_JSON["OpenInv"]:
                    timeOut = openInv["timestamp"]
                    date_timeIni = datetime.datetime.fromtimestamp(timeIni)
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    if date_timeOut - date_timeIni < timeToOpenChest :
                        return True 

## Método para calcular si un jugador ha abierto el menú de habilidades antes de pasarse la primera zona

In [101]:
def getOpenSkill(dict_JSON):
    cont_open = 0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            date_timeIni = datetime.datetime.fromtimestamp(timeIni)
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    for openSkill in dict_JSON["OpenSkill"]:
                        timeOpen = openSkill["timestamp"]
                        date_timeOpen = datetime.datetime.fromtimestamp(timeOpen)
                        if date_timeOpen < date_timeOut and date_timeOpen > date_timeIni :
                            return True
                    return False

## Método para calcular si un jugador ha abierto la tienda la primera vez que vuelve al barco tras pasarse la primera zona

In [ ]:
def getOpenShopAfterCompleteZone(dict_JSON):
    for zone in dict_JSON["LogoutZone"]:
        if zone["zone"] == 11 and zone["next"] == 0:
            timeIni = zone["timestamp"
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["next"] == 0 :
                    timeOut = zoneOut["timestamp"]
                    for openShop in dict_JSON["OpenShop"]:
                        if openShop["timestamp"] < timeOut and openShop["timestamp"] > timeIni:
                            return True

## Método para calcular si un jugador ha usado habilidades al pasarse la primera zona

In [108]:
def getUseSkill(dict_JSON):
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    date_timeIni = datetime.datetime.fromtimestamp(timeIni) 
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    for useSkill in dict_JSON["UseSkill"]:
                        timeUse = useSkill["timestamp"]
                        date_timeUse = datetime.datetime.fromtimestamp(timeUse)
                        if date_timeUse < date_timeOut and date_timeUse > date_timeIni :
                            return True
                    return False

## Método para calcular el número de veces que un jugador se equipa una poción durante la primera zona

In [109]:
def getEquipSkill(dict_JSON):
    cont_equip=0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11:
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeIni = zone["timestamp"]
                    timeOut = zoneOut["timestamp"]
                    for equipSkill in dict_JSON["OpenInv"]:
                        timeEquip = equipSkill["timestamp"]
                        if timeEquip < timeOut and timeEquip > timeIni and equipSkill["equipPot"] :
                            cont_equip+=1
    return cont_equip

## Método para calcular si un jugador usa una poción durante la primera zona

In [110]:
def getUsePot(dict_JSON):
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 :
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    timeIni = zone["timestamp"]
                    for usePot in dict_JSON["UsePot"]:
                        timeUse = usePot["timestamp"]
                        if timeUse < timeOut and timeUse > timeIni :
                            return True
                    return False 

## Método para calcular el tiempo que tarda en cada parte del tutorial cada jugador

In [111]:
def getTimesTutorial(dict_JSON):
    times = []
    i = 0
    for tuto in dict_JSON["TutoTask"]:
        times.insert(i, tuto["timestamp"])
        i += 1
    
    totalTareas = dict_JSON["TutoTask"].length
    i = 0
    fin_times = []
    for t in times:
        if i + 1 <= totalTareas:
            fin_times.insert(i, times[i + 1] - times[i])
            i += 1
    return fin_times

## Método para calcular si el jugador se ha saltado el tutorial en algún punto

In [112]:
def getTutorialOut(dict_JSON):
    if dict_JSON["TutoTask"].length == 0 or dict_JSON["TutoTask"] < 5:
        return True
    else:
        return False